In [5]:
import pandas as pd
import os

# Carregar a matriz de referência de habilidades
# Lendo o arquivo CSV contendo a matriz de referência das habilidades
# O arquivo deve conter colunas como 'Etapa', 'Componente Curricular', 'Descritor', 'Descrição da Habilidade', 'Ciclo 1', 'Ciclo 2', 'Ciclo 3'
df1 = pd.read_csv('Matriz_Referencia_CNCA.csv')

# Transformar o dataframe para um formato longo (melhor para análise)
df1_longo = pd.melt(df1, 
                    value_vars=['Ciclo 1','Ciclo 2','Ciclo 3'],  # Colunas que serão transformadas em linhas
                    id_vars=['Etapa', 'Componente Curricular', 'Descritor', 'Descrição da Habilidade '],  # Colunas mantidas
                    var_name='Ciclos',  # Nome da nova coluna contendo os ciclos
                    value_name='Habilidades')  # Nome da nova coluna contendo os valores

# Remover prefixo 'Ciclo ' dos valores na coluna 'Ciclos'
df1_longo = df1_longo.replace('Ciclo ', '', regex=True)
# Substituir abreviações pelos nomes completos dos componentes curriculares
df1_longo = df1_longo.replace('LPL', 'LÍNGUA PORTUGUESA', regex=True)
df1_longo = df1_longo.replace('MT', 'MATEMÁTICA', regex=True)

# Salvar o dataframe processado em um novo arquivo CSV
df1_longo.to_csv("df1.csv", index=False, encoding="utf-8")

# Definir os ciclos, anos e componentes curriculares a serem processados
ciclos = range(1, 4)  # Ciclos de 1 a 3
anos = range(1, 6)    # Anos de 1 a 5
componentes = ['LPL', 'MT']  # Componentes curriculares

# Lista para armazenar os DataFrames carregados
dataframes = []

# Definir o caminho da pasta onde os arquivos de dados estão armazenados
caminho_pasta = "DadosBrutos"  # Altere conforme necessário

# Percorrer todas as combinações de ciclos, anos e componentes
for ciclo in ciclos:
    for ano in anos:
        for componente in componentes:
            # Construir o nome do arquivo esperado
            nome_arquivo = f"CNCA_CICLO{ciclo}_{ano}ANO_{componente}.csv"
            caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)
            
            # Verificar se o arquivo existe antes de tentar abrir
            if os.path.exists(caminho_arquivo):
                df = pd.read_csv(caminho_arquivo, sep=';')  # Ajustar separador se necessário
                df['Ciclos'] = ciclo
                df['Ano'] = ano
                df['Componente'] = componente
                dataframes.append(df)
                # print(f"Arquivo carregado: {nome_arquivo}")  # Depuração
            else:
                print(f"Arquivo não encontrado: {nome_arquivo}")

# Concatenar todos os DataFrames carregados em um único dataframe
df_concat = pd.concat(dataframes, ignore_index=True)
df_concat.to_csv("df.csv", index=False, encoding="utf-8")

# Filtrar apenas os municípios de interesse
df2 = df_concat[df_concat['Município'].isin(['AQUIRAZ', 'CAUCAIA','EUSEBIO','GUAIUBA','ITAITINGA','MARACANAU','MARANGUAPE','PACATUBA'])]

# Renomear colunas removendo ' (%)' se presente no nome
df2 = df2.rename(columns=lambda x: x.replace(' (%)', '') if ' (%)' in x else x)

# Substituir descrições longas dos anos por versões mais curtas
df2 = df2.replace({
    'ENSINO FUNDAMENTAL DE 9 ANOS - 1º ANO': '1 ANO',
    'ENSINO FUNDAMENTAL DE 9 ANOS - 2º ANO': '2 ANO',
    'ENSINO FUNDAMENTAL DE 9 ANOS - 3º ANO': '3 ANO',
    'ENSINO FUNDAMENTAL DE 9 ANOS - 4º ANO': '4 ANO',
    'ENSINO FUNDAMENTAL DE 9 ANOS - 5º ANO': '5 ANO'
})

# Identificar colunas que começam com 'H', indicando habilidades
colunas_h = [col for col in df2.columns if col.startswith('H')]

# Transformar df2 para formato longo para análise das habilidades
df2_longo = pd.melt(df2, 
                    value_vars=colunas_h,  # Colunas que serão transformadas em linhas
                    id_vars=['Município', 'Componente Curricular', 'Etapa', 'Previstos', 'Avaliados', '% Participação', 
                             'Defasagem', 'Aprendizado intermediário', 'Aprendizado adequado', 'Acerto Total', 'Ciclos'],
                    var_name='Habilidades',  # Nome da nova coluna criada
                    value_name='Percentual de acertos')  # Nome da nova coluna criada

# Salvar o dataframe processado
df2_longo.to_csv("df2.csv", index=False, encoding="utf-8")

# Garantir que a coluna 'Ciclos' seja string para facilitar o merge
df1_longo['Ciclos'] = df1_longo['Ciclos'].astype(str)
df2_longo['Ciclos'] = df2_longo['Ciclos'].astype(str)

# Unir os dois dataframes baseados nos campos comuns
df2_longo_concat = pd.merge(df1_longo, df2_longo, on=['Etapa', 'Componente Curricular', 'Ciclos', 'Habilidades'], how='outer')

# Remover linhas onde a coluna 'Descritor' está vazia
df2_longo_concat = df2_longo_concat.dropna(subset=['Descritor'])

# Filtrar os municípios de interesse no dataframe final
df2_longo_concat = df2_longo_concat[df2_longo_concat['Município'].isin(['AQUIRAZ', 'CAUCAIA','EUSEBIO','GUAIUBA','ITAITINGA','MARACANAU','MARANGUAPE','PACATUBA'])]

# Remover símbolos de porcentagem dos valores
df2_longo_concat = df2_longo_concat.replace('%', '', regex=True)
df2_longo_concat = df2_longo_concat.rename(columns=lambda x: x.replace('% ', '') if '% ' in x else x)

# Salvar o dataframe final
df2_longo_concat.to_csv("df_final.csv", index=False, encoding="utf-8")